In [1]:
import pandas as pd
import numpy as np

In [2]:
def GetData(df):
    cols = df.columns[:-1]
    X = df[cols].to_numpy()
    Y=df['signal'].to_numpy().reshape((len(df),1))
    return X,Y

xtrain, ytrain = GetData(pd.read_csv("train_bonus.csv"))
xtest, ytest   = GetData(pd.read_csv("test_bonus.csv"))

print(xtrain.shape, ytrain.shape)

(20000, 7) (20000, 1)


In [3]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

def SimpleScore(pred, ytest):
    if (pred.shape != ytest.shape): 
        pred=pred.reshape(ytest.shape) # important! prevents broadcasting
    diff = np.abs(pred - np.clip(ytest,0,1))
    return np.mean(np.less(diff,0.5))  

In [4]:
# this is a start point of your analysis, ... but not a good one
model = Sequential([
    layers.Dense(100, activation="sigmoid"),
    layers.Dense(100, activation="sigmoid"),
    layers.Dense(1, activation="sigmoid")
    ])

model.compile(optimizer="adam", loss='binary_crossentropy') 
model.fit(xtrain, ytrain, epochs=30, verbose=1, batch_size=16, validation_data=(xtest, ytest), callbacks=[early_stop])

print("test score", SimpleScore(model.predict(xtest),ytest), "train score", SimpleScore(model.predict(xtrain),ytrain))

Epoch 1/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6977 - val_loss: 0.6938
Epoch 2/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6958 - val_loss: 0.6961
Epoch 3/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6960 - val_loss: 0.6941
Epoch 4/30
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6952 - val_loss: 0.6931
Epoch 5/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6954 - val_loss: 0.6931
Epoch 6/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6951 - val_loss: 0.6944
Epoch 7/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6946 - val_loss: 0.6932
Epoch 8/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6940 - val_loss: 0.6930
Epoch 9/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6941 - val_loss: 0.6939
Epoch 10/30
1250/1250 [==============================] - 3s 2ms/step - loss: 0.693

## I wasn't very successfull

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

input_len = 7

def buildCustomModel(input_shape, layer_number=[100, 100, 100], act_method=['relu', 'relu', 'relu', 'sigmoid'], batchnormalize= True, optimizer="adam", loss='mean_squared_error'):
    m_list = []
    m_list.append(layers.InputLayer(input_shape=input_shape))
    
    for i in range(len(layer_number)):
        m_list.append(layers.Dense(layer_number[i], activation=act_method[i]))
        if batchnormalize:
            m_list.append(layers.BatchNormalization(axis=1))
    m_list.append(layers.Dense(1, activation= act_method[-1]))

    model = keras.Sequential(m_list)
    model.compile(optimizer=optimizer, loss=loss)
    return model

param_grid = [
    {
    'input_shape': [input_len], 
    'layer_number': [[1000, 750, 500, 250], [750, 500, 250, 100], [500, 500, 500, 500]],
    'act_method': [['relu', 'relu', 'relu', 'relu', 'sigmoid']], 
    'batchnormalize':[True],
    'optimizer': ['adam'], 
    'loss': ['binary_crossentropy']
    }
    ]


model = KerasClassifier(build_fn=buildCustomModel, verbose=0)
gs = GridSearchCV(estimator=model,param_grid=param_grid, cv=2, n_jobs=None, scoring='accuracy', verbose=2)
fitted_model = gs.fit(xtrain, ytrain, epochs= 20)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


C:\Users\Markus\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END act_method=['relu', 'relu', 'relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=7, layer_number=[1000, 750, 500, 250], loss=binary_crossentropy, optimizer=adam; total time=  38.5s


C:\Users\Markus\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END act_method=['relu', 'relu', 'relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=7, layer_number=[1000, 750, 500, 250], loss=binary_crossentropy, optimizer=adam; total time=  38.4s


...

[CV] END act_method=['relu', 'relu', 'relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=7, layer_number=[750, 500, 250, 100], loss=binary_crossentropy, optimizer=adam; total time=  39.3s


C:\Users\Markus\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END act_method=['relu', 'relu', 'relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=7, layer_number=[500, 500, 500, 500], loss=binary_crossentropy, optimizer=adam; total time=  39.3s


C:\Users\Markus\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END act_method=['relu', 'relu', 'relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=7, layer_number=[500, 500, 500, 500], loss=binary_crossentropy, optimizer=adam; total time=  39.1s


In [9]:
fitted_model.best_score_

0.5491999999999999

In [8]:
model = Sequential([
    layers.InputLayer(input_shape=input_len),
    layers.Dense(500, activation='relu'),
    layers.BatchNormalization(axis=1),
    layers.Dense(500, activation='relu'),
    layers.BatchNormalization(axis=1),
    layers.Dense(500, activation='relu'),
    layers.BatchNormalization(axis=1),
    layers.Dense(500, activation='relu'),
    layers.BatchNormalization(axis=1),
    layers.Dense(1, activation='sigmoid'),
    ])

model.compile(optimizer="adam", loss='binary_crossentropy') 
model.fit(xtrain, ytrain, epochs=100, verbose=1, batch_size=16, validation_data=(xtest, ytest), callbacks=[early_stop])

print("test score", SimpleScore(model.predict(xtest),ytest), "train score", SimpleScore(model.predict(xtrain),ytrain))

Epoch 1/100
1250/1250 [==============================] - 9s 7ms/step - loss: 0.7680 - val_loss: 0.7058
Epoch 2/100
1250/1250 [==============================] - 8s 7ms/step - loss: 0.7100 - val_loss: 0.7377
Epoch 3/100
1250/1250 [==============================] - 8s 7ms/step - loss: 0.7060 - val_loss: 0.6964
Epoch 4/100
1250/1250 [==============================] - 9s 7ms/step - loss: 0.7037 - val_loss: 0.6955
Epoch 5/100
1250/1250 [==============================] - 8s 7ms/step - loss: 0.7026 - val_loss: 0.7002
Epoch 6/100
1250/1250 [==============================] - 8s 7ms/step - loss: 0.6997 - val_loss: 0.6951
Epoch 7/100
1250/1250 [==============================] - 9s 7ms/step - loss: 0.6980 - val_loss: 0.6955
Epoch 8/100
1250/1250 [==============================] - 9s 8ms/step - loss: 0.6968 - val_loss: 0.7037
Epoch 9/100
1250/1250 [==============================] - 9s 7ms/step - loss: 0.6957 - val_loss: 0.6964
Epoch 10/100
1250/1250 [==============================] - 9s 7ms/step - l